Ab Roller Counter

Notes: Use OpenCV and MediaPipe to draw landmarks and connections, this will be based off that push-up counter project.

Make the rep counter valid only to a certain point of view, add instructions for the user to position the streaming device in a certan angle where the body is e.g. sideways to mantain those 2d angles.

Possibly improve to more angles once the initial goal is accomplished.